#@markdown <font size='+2' color='#005F6A'>**ButtonBox**</font><br>

#@markdown * An alternative for the Tab widget for selecting **large amount of items**

#@markdown * Uses **Buttons** and a **Box** with **flex wrap layout**.

#@markdown * Provide a **mechanism for select, append and delete** an item .

#@markdown <table><tr><td><font size='+2'>

#@markdown

#@markdown ```python

#@markdown def showSelection (bbox):

#@markdown     try:    textarea.value = plain (render_doc(bbox.button.tooltip))

#@markdown     except: textarea.value = 'error calling render_doc with '+bbox.button.tooltip

#@markdown #

#@markdown textarea  = Textarea  (layout=Layout(width='1000px', height='300px'))

#@markdown buttonbox = ButtonBox (__builtins__.__dict__.keys(), clicker=showSelection)

#@markdown display (buttonbox.box, textarea)

#@markdown ```

from ipywidgets import Button, Box, Layout, Textarea



class ButtonBox ():

  # An alternative for the Tab widget

  def _clicker (self, b):

    # search clicked button and remember

    self.position, self.button = [(i,but) for i, but in enumerate(self.buttons) if b == but][0]



    # unselect (color) all buttons and select new (list of colors here: https://www.quackit.com/css/css_color_codes.cfm)

    self.unselect()

    self.buttons[self.position].style={'button_color':self.color}



    # fire event

    if self.clicker: self.clicker (self)



  def append (self, description, select=True):

    # add new selctor button at the end

    b = Button (description = description if len (description) <= self.maxchar else f'{description[:(self.maxchar-3)]}...',

                layout      = Layout(width='auto', height='21px'),

                tooltip     = f'{description}')

    self.buttons.append(b)

    self.box.children = [*self.box.children, b]



    # select new button / bind event

    if select:

      self.button, self.position = b, len(self.buttons) - 1

      self._clicker (b)

    b.on_click (self._clicker)



  def remove (self, position=None):

    # remove selected if no position given

    if not position: position = self.position

    if not position or not position < len(self.buttons) or position < 0: return



    # clean up

    self.box.children = [*self.box.children[:position], *self.box.children[position+1:]]

    self.buttons = [*self.buttons[:position], *self.buttons[position+1:]]

    self.button, self.position = None, -1



  def unselect (self):

    # unselect all buttons

    for b in self.buttons: b.style={'button_color':None}

  def remove_all(self):
    self.box.children = []
    self.buttons = []
    self.button, self.position = None, -1


  def __init__ (self, descriptions, clicker=None, maxchar=60, color='powderblue'):

    # remember stuff - list to set

    self.descriptions = descriptions if len (descriptions) == len (set (descriptions)) else set (descriptions)

    self.clicker, self.maxchar, self.color, self.position, self.button = clicker, maxchar, color, -1, None



    # make buttons

    self.buttons = [Button (description = i if len (i) <= maxchar else f'{i[:(maxchar-3)]}...',

                            layout      = Layout(width='auto', height='21px'),

                            tooltip     = f'{i}')

                    for i in descriptions]



    # put them in a box / bind event

    self.box = Box (layout   = Layout (display='flex', flex_flow='wrap'),

                    children = self.buttons)

    for button in self.buttons: button.on_click (self._clicker)



# alternative:

# def showSelection (bbox):

#     try:    textarea.value = eval (bbox.button.tooltip+".__doc__")

#     except: textarea.value = 'error calling .__doc__ '+bbox.button.tooltip

# textarea  = Textarea  (layout=Layout(width='1000px', height='300px'))

# buttonbox = ButtonBox (locals().keys(), clicker=showSelection)

# display (buttonbox.box, textarea)

# *******************************************************************************************************************************************************************************************

# ****usage**********************************************************************************************************************************************************************************

# *******************************************************************************************************************************************************************************************

from ipywidgets   import Layout, Textarea, Button, Output, VBox

from huggingface_hub import HfApi, list_datasets

from IPython.display import display, HTML

import datetime

api = HfApi()
output_area = Output()

def get_popular_datasets(sort_by="downloads", limit=50):
    """Fetches popular datasets from HuggingFace Hub."""
    with output_area:
        print(f"Fetching datasets sorted by {sort_by}...")
    if sort_by == "trending_score":
        datasets = list(api.list_datasets(limit=200, expand=["trendingScore"]))
        datasets.sort(key=lambda x: x.trendingScore if x.trendingScore is not None else -1, reverse=True)
    else: # "downloads" or "likes"
        datasets = list(api.list_datasets(sort=sort_by, direction=-1, limit=limit, expand=["downloads", "downloadsAllTime"]))
    return datasets[:limit]

def show_dataset_details(bbox):
    """Displays detailed information about the selected dataset."""
    with output_area:
        output_area.clear_output()
        dataset_id = bbox.button.tooltip
        try:
            dataset_info = api.get_dataset_info(repo_id=dataset_id, expand=["downloads", "downloadsAllTime"])
            
            details = f"<b>Dataset:</b> {dataset_info.id}<br>"
            details += f"<b>Downloads (All Time):</b> {dataset_info.downloadsAllTime:,}<br>"
            details += f"<b>Downloads (Last Month):</b> {dataset_info.downloads:,}<br>" # 'downloads' field is typically last 30 days
            details += f"<b>Likes:</b> {dataset_info.likes:,}<br>"
            if hasattr(dataset_info, 'trendingScore'):
                details += f"<b>Trending Score:</b> {dataset_info.trendingScore}<br>"
            
            # Add a link to the HuggingFace page
            hf_link = f"https://huggingface.co/datasets/{dataset_id}"
            details += f"<br><a href='{hf_link}' target='_blank'>View on HuggingFace Hub</a>"
            
            textarea.value = details
        except Exception as e:
            textarea.value = f"Error fetching details for {dataset_id}: {e}"

def update_dataset_list(filter_bbox):
    """Updates the main dataset ButtonBox based on the selected filter."""
    selected_filter = filter_bbox.button.description
    
    # Clear existing buttons
    dataset_buttonbox.remove_all()
    
    sort_param = "downloads"
    if selected_filter == "Total Downloads":
        sort_param = "downloads"
    elif selected_filter == "Trending":
        sort_param = "trending_score"
    elif selected_filter == "Likes":
        sort_param = "likes"
    
    popular_datasets = get_popular_datasets(sort_by=sort_param)
    
    for dataset in popular_datasets:
        dataset_buttonbox.append(dataset.id, select=False)

# UI Elements
textarea = Textarea(layout=Layout(width='100%', height='200px'))

filter_options = ["Total Downloads", "Trending", "Likes"]
filter_buttonbox = ButtonBox(filter_options, clicker=update_dataset_list)

dataset_buttonbox = ButtonBox([], clicker=show_dataset_details)

# Initial display
display(filter_buttonbox.box, dataset_buttonbox.box, textarea, output_area)

# Trigger initial load
# Manually simulate a click on the first filter option to load datasets
if filter_buttonbox.buttons:
    filter_buttonbox._clicker(filter_buttonbox.buttons[0])
